In [4]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
IV127 = Path('/content/drive/My Drive/IV127')

In [6]:
!pip install transformers
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [9]:
device='cpu'

def load_model():
    model = AutoModelForMaskedLM.from_pretrained('xlm-roberta-base').to(device)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
    return tokenizer, model

tokenizer, model = load_model()

def score(sentence, word):
    num_masks = len(tokenizer.tokenize(word))
    labels = sentence.replace('_', word)
    mask = sentence.replace('_', '<mask>' * num_masks)

    tensor_labels = torch.tensor(tokenizer.encode(labels)).unsqueeze(0).to(device)
    tensor_mask = torch.tensor(tokenizer.encode(mask)).unsqueeze(0).to(device)
    with torch.no_grad():
        loss = model(tensor_mask, labels = tensor_labels)[0]
    return np.exp(loss.item())

def get_prediction(row):
    return row.solutions_agg[np.array([score(row.sentence, sol) for sol in row.solutions_agg]).argmin()]

Some weights of XLMRobertaForMaskedLM were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# device = 'cpu'
# model_id = 'gpt2'
# model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
# tokenizer = GPT2Tokenizer.from_pretrained(model_id)

In [13]:
sentence = "The _ is at the end of the hall."
word = 'bathroom'

input_ = tokenizer(sentence.replace("_", word), return_tensors='pt')
w = len(tokenizer.encode(sentence.replace('_', word)))
wout = len(tokenizer.encode(sentence))
num_masks = w - wout + 1

labels = sentence.replace('_', word)
mask = sentence.replace('_', 'mask' * num_masks)
print(labels)
print(mask)
tensor_labels = torch.tensor(tokenizer.encode(labels)).unsqueeze(0).to(device)
tensor_mask = torch.tensor(tokenizer.encode(mask)).unsqueeze(0).to(device)

The bathroom is at the end of the hall.
The mask is at the end of the hall.


In [ ]:
tokenizer(sentence, return_tensors='pt')

{'input_ids': tensor([[ 464, 4808,  318,  379,  262,  886,  286,  262, 6899,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
input_

{'input_ids': tensor([[  464, 12436,   318,   379,   262,   886,   286,   262,  6899,    13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
tensor_labels

tensor([[     0,    581, 157796,     83,     99,     70,   3564,    111,     70,
          23664,      5,      2]])

In [15]:
tensor_mask

tensor([[    0,   581,  7021,    83,    99,    70,  3564,   111,    70, 23664,
             5,     2]])